<a href="https://colab.research.google.com/github/Evgeniya371/PRA3024/blob/main/PyCharm_inensity_coefficients.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import intensity_coefficients_functions as icf
from coating import substrate, layers
import sympy

# Provide the input values below and run

# Input values
angle_1 = 0  # [degrees]          # Angle of incidence (with respect to the normal to the surface)
refraction_index_1 = 1            # Refraction index of the incidence medium (usually air)
polarization = 's'                # Select the polarization by specifying the input as 's' or 'p'
wavelength = 1000  # [nm]         # Wavelength of the incident light in nanometers [nm]

# !!!
# Materials are specified in the 'material_parameters.py' file
# The stack of layers is defined in the 'coating.py' file


# Compute the matrices and the intensity coefficients
T, R, M_stack, r, t, M_reversed_stack, r_reversed, t_reversed = icf.intensity_coefficients(stack_list = layers, lambda_o = wavelength, d_sub = substrate.thickness, n_sub = substrate.refractive_index, n_1 = refraction_index_1, phi_1 = angle_1, polarization=polarization)

# Print the output
print('')
print('')
print('')
print('')
print('Computed matrices and intensity coefficients:')
print('')
print('Matrix, stack:')
print('')
M = sympy.Matrix(M_reversed_stack)
sympy.pprint(M)
print('')
print('Reflection coefficient: ', r)
print('Transmission coefficient: ', t)
print('')
print('Matrix, reversed stack:')
print('')
M = sympy.Matrix(M_stack)
sympy.pprint(M)
print('')
print('Reflection coefficient, reversed stack: ', r_reversed)
print('Transmission coefficient, reversed stack: ', t_reversed)
print('')
print('Intensity coefficients:')
print('T:', T)
print('R:', R)
print('T+R:', T+R)


In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sympy
from sympy import Matrix, symbols, latex
import matplotlib
import sys


# This function computes a single film matrix
# s-polarization

def single_film_matrix_s(n, o_t, lambda_o, phi):
# n - refraction index of the layer
# o_t - optical thickness of the layer (expressed as a fraction of the wavelength of light)
# lambda_o - wavelength of the incident light (in meters)
# phi - propagation angle (in radians)

    k_0 = 2*np.pi/lambda_o              # Wave number: k=2*pi/wavelength
    optical_thickness = o_t * lambda_o    # Optical thickness = n * d = o_t * lambda_o, where d is geometrical distnace

    # Calculation of the matrix elements:
    # Formulas are from: Stenzel, O. Optical Coatings, vol. 54 (2014)
    m_11 = np.around(np.cos(k_0 * optical_thickness * np.cos(phi)), decimals=5)
    m_12 = - 1j / (n * np.cos(phi))* np.around(np.sin(k_0 * optical_thickness * np.cos(phi)), decimals=5)
    m_21 = - 1j * n * np.cos(phi) * np.around(np.sin(k_0 * optical_thickness * np.cos(phi)), decimals=5)
    m_22 = np.around(np.cos(k_0 * optical_thickness *np.cos(phi)), decimals=5)

    # Assign the calculated values to the matrix elements
    matrix=np.empty((2, 2), dtype=object)
    matrix[0, 0] = m_11
    matrix[0, 1] = m_12
    matrix[1, 0] = m_21
    matrix[1, 1] = m_22
    return matrix



# This function computes a single film matrix
# p-polarization

def single_film_matrix_p(n, o_t, lambda_o, phi):
# n - refraction index of the layer
# o_t - optical thickness of the layer (expressed as a fraction of the wavelength of light)
# lambda_o - wavelength of the incident light (in meters)
# phi - propagation angle (in radians)

    k_0 = 2 * np.pi/lambda_o              # Wave number: k=2*pi/wavelength
    optical_thickness = o_t * lambda_o    # Optical thickness = n * d = o_t * lambda_o, where d is geometrical distnace

    # Calculation of the matrix elements:
    # Formulas are from: Stenzel, O. Optical Coatings, vol. 54 (2014)
    m_11 = np.around(np.cos(k_0 * optical_thickness * np.cos(phi)), decimals=5)
    m_12 = - 1j * n / np.cos(phi) * np.around(np.sin( k_0 * optical_thickness * np.cos(phi) ), decimals=5)
    m_21 = - 1j * np.cos(phi) / n * np.around(np.sin( k_0 * optical_thickness * np.cos(phi) ), decimals=5)
    m_22 = np.around(np.cos( k_0 * optical_thickness * np.cos(phi) ), decimals=5)

    # Assign the calculated values to the matrix elements
    matrix = np.empty((2, 2), dtype=object)
    matrix[0, 0] = m_11
    matrix[0, 1] = m_12
    matrix[1, 0] = m_21
    matrix[1, 1] = m_22
    return matrix



# This function computes reflection coefficient
# s-polarization

def reflection_coefficient_s(phi_1, phi_sub, n_1, n_sub, M):
# phi_1 - propagation angle in the incidence medium
# phi_sub - propagation angle in the substrate
# n_1 - refraction index  of the incidence medium
# n_sub - refraction index of the substrate material
# M - coating matrix


    # Matrix elements
    m_11 = M[0, 0]
    m_12 = M[0, 1]
    m_21 = M[1, 0]
    m_22 = M[1, 1]

    # Calculation of the reflection coefficient:
    # Formula is taken from: Stenzel, O. Optical Coatings, vol. 54 (2014)
    r = ((m_11 + m_12 * n_sub * np.cos(phi_sub)) * n_1 * np.cos(phi_1) - (m_21 + m_22 * n_sub * np.cos(phi_sub))) / ((m_11 + m_12 * n_sub * np.cos(phi_sub)) * n_1 * np.cos(phi_1) + m_21 + m_22 * n_sub * np.cos(phi_sub))

    return r



# This function computes reflection coefficient
# p-polarization

def reflection_coefficient_p(phi_1, phi_sub, n_1, n_sub, M):
# phi_1 - propagation angle in the incidence medium
# phi_sub - propagation angle in the substrate
# n_1 - refraction index  of the incidence medium
# n_sub - refraction index of the substrate material
# M - coating matrix


    # Matrix elements
    m_11 = M[0, 0]
    m_12 = M[0, 1]
    m_21 = M[1, 0]
    m_22 = M[1, 1]

    # Calculation of the reflection coefficient:
    # Formula is taken from: Stenzel, O. Optical Coatings, vol. 54 (2014)
    r = ((m_11 + m_12 * np.cos(phi_sub) / n_sub) * np.cos(phi_1) / n_1 - (m_21 + m_22 * np.cos(phi_sub) / n_sub)) / ((m_11 + m_12 * np.cos(phi_sub) / n_sub) * np.cos(phi_1)/n_1 + m_21 + m_22 * np.cos(phi_sub) / n_sub)

    return r


# This function computes transmission coefficient
# s-polarization

def transmission_coefficient_s(phi_1, phi_sub, n_1, n_sub, M):
# phi_1 - propagation angle in the incidence medium
# phi_sub - propagation angle in the substrate
# n_1 - refraction index  of the incidence medium
# n_sub - refraction index of the substrate material
# M - coating matrix


    # Matrix elements
    m_11 = M[0, 0]
    m_12 = M[0, 1]
    m_21 = M[1, 0]
    m_22 = M[1, 1]

    # Calculation of the reflection coefficient:
    # Formula is taken from: Stenzel, O. Optical Coatings, vol. 54 (2014)
    t = (2 * n_1 * np.cos(phi_1)) / ((m_11 + m_12 * n_sub * np.cos(phi_sub)) * n_1 * np.cos(phi_1) + m_21 + m_22 * n_sub * np.cos(phi_sub))

    return t



# This function computes transmission coefficient
# p-polarization

def transmission_coefficient_p(phi_1, phi_sub, n_1, n_sub, M):
# phi_1 - propagation angle in the incidence medium
# phi_sub - propagation angle in the substrate
# n_1 - refraction index  of the incidence medium
# n_sub - refraction index of the substrate material
# M - coating matrix


    # Matrix elements
    m_11 = M[0, 0]
    m_12 = M[0, 1]
    m_21 = M[1, 0]
    m_22 = M[1, 1]

    # Calculation of the reflection coefficient:
    # Formula is taken from: Stenzel, O. Optical Coatings, vol. 54 (2014)
    t = (2 * np.cos(phi_1) / n_1) / ((m_11 + m_12 * np.cos(phi_sub) / n_sub) * np.cos(phi_1)/n_1 + m_21 + m_22 * np.cos(phi_sub) / n_sub)

    return t



# This function computes the M_stack matrix and reflection coefficient of the multilayer coating

def stack(stack_list, lambda_o, phi_1, n_1, n_sub, polarization):
# stack_list - list of layers that represents a stack of coating layers
# lambda_o - wavelength of the incident light
# phi_1 - propagation angle in the incidence medium
# phi_sub - propagation angle in the substrate
# n_1 - refraction index  of the incidence medium
# n_sub - refraction index of the substrate material
# polarization - polarization of the light wave ('s' or 'p')


    M_stack = np.identity(2) # Initialization of the M_stack matrix. Identity matrix is chosen as the intial matrix, since multiplication by identity matrix does not change the mutliplied matrix.
    lambda_o = lambda_o * 10 ** (-9)   # Conversion from nanometers to meters
    phi_1 = math.radians(phi_1)        # Conversion to radians

    ang_prev = phi_1   # Initialization of the angle_previous variable - propagation in the first medium
    n_prev = n_1       # Initialization of the n_previous variable - propagation in the first medium

    layers_dict = stack_of_layers(stack_list)  # Dictionary that represents a stack of coating layers

    print('')
    print('The propagation of the light beam in each layer:')
    print('')

    # Iteration over the supplied layers_dict dictionary

    for k, v in layers_dict.items():
        print(k) # Print the current layer
        refractive_index = v.refractive_index    # The refractive index of the current layer is selected in this line
        optical_thickness = v.optical_thickness  # The optical thickness of the current layer is selected in this line
        angle = angle_of_refraction(angle_previous = ang_prev, n_previous = n_prev, n_current = refractive_index) # Each successive angle in the layers is computed using Snell's law
        if angle == None:
           sys.exit()
        geometrical_thickness_m = optical_thickness*lambda_o/refractive_index # Geometical thickness = optical thickness * wavelength / refractive index
        geometrical_thickness_nm = geometrical_thickness_m/(10 ** (-9)) # Conversion from meters to nanometers
        print('The optimal geometrical thickness:', geometrical_thickness_nm, 'nm') # Print the optimal geometrical thickness


        # If input value is 's', compute s-polarization
        if polarization == 's':
           M = single_film_matrix_s(n = refractive_index, o_t = optical_thickness, lambda_o = lambda_o, phi = angle)
           M_stack = np.matmul(M_stack, M)  # Matrix multiplication for the successive layers

        # If input value is 'p', compute p-polarization
        if polarization == 'p':
           M = single_film_matrix_p(n = refractive_index, o_t = optical_thickness, lambda_o = lambda_o, phi = angle)
           M_stack = np.matmul(M_stack, M)  # Matrix multiplication for the successive layers

        M_stack = M_stack

        # Update the values of the angle_previous and n_previous variables

        if phi_1 == 0:  # If the incident angle is 0, all propagation angles in the multilayer coating are 0
           ang_prev = 0
        else:
           ang_prev =  np.pi/2 - angle # In the right triangle the sum of the two acute angles is pi/2

        n_prev = refractive_index

    # Calculate the propagation angle of the final layer - substrate
    phi_sub = angle_of_refraction(angle_previous = ang_prev, n_previous = n_prev, n_current = n_sub)


    # If input value is 's', compute s-polarization
    if polarization == 's':
       r = reflection_coefficient_s(phi_1, phi_sub, n_1, n_sub, M_stack)
       t = transmission_coefficient_s(phi_1, phi_sub, n_1, n_sub, M_stack)

    # If input value is 'p', compute p-polarization
    if polarization == 'p':
       r = reflection_coefficient_p(phi_1, phi_sub, n_1, n_sub, M_stack)
       t = transmission_coefficient_p(phi_1, phi_sub, n_1, n_sub, M_stack)


    # Computation of the characheristic matrix of the stack in the reversed order

    M_reversed_stack = np.empty((2, 2), dtype=object) # Initialisation of the reversed stack matrix

    M_reversed_stack[0, 0] = M_stack[1, 1]
    M_reversed_stack[0, 1] = M_stack[0, 1]
    M_reversed_stack[1, 0] = M_stack[1, 0]
    M_reversed_stack[1, 1] = M_stack[0, 0]


    # If input value is 's', compute s-polarization
    if polarization == 's':
       r_reversed = reflection_coefficient_s(phi_1 = phi_sub, phi_sub = phi_1, n_1 = n_sub, n_sub = n_1, M = M_reversed_stack)
       t_reversed = transmission_coefficient_s(phi_1 = phi_sub, phi_sub = phi_1, n_1 = n_sub, n_sub = n_1, M = M_reversed_stack)

    # If input value is 'p', compute p-polarization
    if polarization == 'p':
       r_reversed = reflection_coefficient_p(phi_1 = phi_sub, phi_sub = phi_1, n_1 = n_sub, n_sub = n_1, M = M_reversed_stack)
       t_reversed = transmission_coefficient_p(phi_1 = phi_sub, phi_sub = phi_1, n_1 = n_sub, n_sub = n_1, M = M_reversed_stack)


    return M_stack, r, t, M_reversed_stack, r_reversed, t_reversed




# This funciton finds the reflection and transmission coefficients for the "incidence medium & substrate" system
# Here, r_1_s, t_1_s are r_13 and t_13, respectively. If it is neccessary to compute r_31 and t_31, reverse the refractive indices in the input

def coefficients_media_1_sub(phi_1, n_1, n_sub, polarization):
# phi_1 - propagation angle in the incidence medium
# n_1 - refraction index  of the incidence medium
# n_sub - refraction index of the substrate material
# polarization - polarization of the light wave ('s' or 'p')

    phi_1 = math.radians(phi_1)  # Conversion to radians

    # If input value is 's', compute s-polarization
    if polarization == 's':
       r_1_s = (n_1 * np.cos(phi_1) -  n_sub * math.sqrt(1-(n_1 * np.sin(phi_1) / n_sub)**2)) / (n_1 * np.cos(phi_1) +  n_sub * math.sqrt(1- (n_1 * np.sin(phi_1)/n_sub)**2))
       t_1_s = 2 * n_1 * np.cos(phi_1) / (n_1 * np.cos(phi_1) +  n_sub * math.sqrt(1 - (n_1 * np.sin(phi_1)/n_sub)**2))

    # If input value is 'p', compute p-polarization
    if polarization == 'p':
      r_1_s = (n_1 * math.sqrt(1-(n_1 * np.sin(phi_1)/n_sub)**2) - n_sub * np.cos(phi_1)) / (n_1 * math.sqrt(1- (n_1 * np.sin(phi_1)/n_sub)**2) + n_sub * np.cos(phi_1))
      t_1_s = 2 * n_1 * np.cos(phi_1) / (n_1 * math.sqrt(1 - (n_1 * np.sin(phi_1)/n_sub)**2) + n_sub * np.cos(phi_1))

    return r_1_s, t_1_s



# This function computes intensity coefficients T and R

def intensity_coefficients(stack_list, lambda_o, d_sub, n_sub, n_1, phi_1, polarization):
# stack_list - list of layers that represents a stack of coating layers
# lambda_o - wavelength of the incident light
# d_sub - thickness of the substrate
# n_sub - refraction index of the substrate material
# n_1 - refraction index  of the incidence medium
# phi_1 - propagation angle in the incidence medium
# polarization - polarization of the light wave ('s' or 'p')

    c = 3 * 10**8                 # Speed of light [m/s]
    nu = c/(lambda_o * 10**(-9))  # Frequency of light [Hz]

    # Function calls
    M_stack, r, t, M_reversed_stack, r_reversed, t_reversed = stack(stack_list, lambda_o, phi_1, n_1, n_sub, polarization)
    r_1_s, t_1_s = coefficients_media_1_sub(phi_1 = phi_1, n_1 = n_sub, n_sub = n_1, polarization = polarization)

    t_123 = t # Transmission coefficient t_123 of the M_stack
    r_123 = r # Reflection coefficient r_123 of the M_stack
    t_321 = t_reversed # Transmission coefficient t_321 of the reversed M_stack
    r_321 = r_reversed # Reflection coefficient r_321 of the reversed M_stack
    r_31 = r_1_s # Substrate -> incidence medium reflection coefficient
    t_31 = t_1_s # Substrate -> incidence medium transmission coefficient
    #print('r_31:', r_31, 't_31:', t_31) # Uncomment for the control

    # Intensity coefficients
    # Formulas are from: Stenzel, O. Optical Coatings, vol. 54 (2014)
    T = (abs(t_123)**2 * abs(t_31)**2 * math.exp(-4 * np.pi * nu * d_sub * np.imag(math.sqrt(n_sub**2 - n_1**2 * np.sin(phi_1)**2))))/(1 - abs(r_321)**2 * abs(r_31)**2 * math.exp(-8 * np.pi * nu * d_sub * np.imag(math.sqrt(n_sub**2 - n_1**2 * np.sin(phi_1)**2))))
    R = abs(r_123)**2 + (abs(t_123)**2 * abs(r_31)**2 * abs(t_321)**2 * math.exp(-8 * np.pi * nu * d_sub * np.imag(math.sqrt(n_sub**2 - n_1**2 * np.sin(phi_1)**2))))/(1 - abs(r_321)**2 * abs(r_31)**2 * math.exp(-8 * np.pi * nu * d_sub * np.imag(math.sqrt(n_sub**2 - n_1**2 * np.sin(phi_1)**2))))

    return T, R, M_stack, r, t, M_reversed_stack, r_reversed, t_reversed


# This function finds the propagation angle in the current medium when the propagation angle of the previous medium and
# The refraction indices of the previous/current medium are provided
# The angle is calculated using Snell's law: n_previous * sin(angle_previous) = n_current * sin(angle)

def angle_of_refraction(angle_previous, n_previous, n_current):
# angle_previous - propagation angle in the previous medium (e.g., in the previous layer of the multilayer coating) (in radians!)
# n_previous - refractive index of the previous medium (e.g., of the previous layer of the multilayer coating)
# n_current - refractive index of the current medium (e.g., of the current layer of the multilayer coating)

    print('The previous propagation angle:', angle_previous, 'rad')

    # Determine the critical angle of refraction: sin(angle_crit) = n_current/n_previous * sin(90) = n_current/n_previous

    # Check this condition: n_previous > n_current
    if n_previous > n_current:
       print('n_previous > n_current: the condition for the total reflection should be checked!')
       # Calculate the sin of critical angle
       sin_of_angle_crit = n_current/n_previous      # sin(angle_crit) = n_current/n_previous
       # Check if the calclated value is valid
       if (sin_of_angle_crit < -1) or (sin_of_angle_crit > 1):
           print('sin(angle_crit) is an invalid value! sin is a bounded function: -1 < sin(angle_crit) < 1 ')
           return None
       # Calculate the critical angle
       angle_crit = np.arcsin(sin_of_angle_crit)     # angle_crit = arcsin(sin(angle_crit)) = arcsin(n/n_previous)
       print('The critical angle:', angle_crit, 'rad')

        ### The following if statement checks 3 cases ###

       # Check the condition for the total internal refelection
       if angle_previous > angle_crit: # Total internal refelection
           print('Total internal reflection: the current angle is equal to', -angle_previous, 'rad')
           print('Execution is terminated. No forward propagation through the layers due to the total internal reflection')
           return None


       # Check the condition for the limiting case
       elif angle_previous == angle_crit: # Critical angle
           angle_of_refraction  = - angle_previous   # Total internal refelection
           print('The limiting case: the previous angle is equal to the critcal angle. Therefore, the current angle is equal to π\2 rad')
           print('Execution is terminated. No forward propagation through the layers due to the critial angle case')
           return None

       # If the previous angle is smaller than the critical angle, no total internal reflection occurs.  Continue the calculation of the refraction angle
       else:
         print('The condition is checked: no total internal reflection')
         # Calculate the sin of the refraction angle
         sin_of_angle = n_previous * np.sin(angle_previous)/n_current # Calculate the sin of the refraction angle
         # Check if the calclated value is valid
         if ( sin_of_angle < -1) or ( sin_of_angle > 1):
             print('sin(angle_of_refraction) is an invalid value! sin is a bounded function: -1 < sin(angle_of_refraction) < 1 ')
             return None
         # Calculate the refraction angle
         angle_of_refraction  = np.arcsin(sin_of_angle)
         print('The current propagation angle:', angle_of_refraction, 'rad')
         return angle_of_refraction # Output is in radians!

    # If n_previous is not greater than n_current, calulate the refraction angle
    else:
       # Calculate the sin of the refraction angle
       sin_of_angle = n_previous * np.sin(angle_previous)/n_current
       # Check if the calclated value is valid
       if ( sin_of_angle < -1) or ( sin_of_angle > 1):
           print('sin(angle_of_refraction) is an invalid value! sin is bounded function: -1 < sin(angle_of_refraction) < 1 ')
           return None
       # Calculate the refraction angle
       angle_of_refraction  = np.arcsin(sin_of_angle)
       print('The current propagation angle:', angle_of_refraction, 'rad')
       return angle_of_refraction # Output is in radians!

def stack_of_layers(stack_list):
    counter = 1
    layers = {}
    for i in stack_list:
        string = 'Layer # {}'.format(counter)
        layers[string] = i
        counter = counter + 1
    return layers


In [ ]:
from Materials import Materials

# Create the instances of the class Material. Initialise the materials in the way shown below:

# The values of parameters are found in Granata et al, 2020)
fused_SiO2 = Materials()
fused_SiO2.name = 'fused SiO2'            # Name of the material
fused_SiO2.Young_modulus = 71.2*10**9     # Young's modulus (Pa)
fused_SiO2.Poisson_ratio = 0.16           # Poisson ratio
fused_SiO2.refractive_index = 1.45        # Refractive index
fused_SiO2.mass_density = 2220            # Mass density (kg/m^3)

SiO2 = Materials()
SiO2.name = 'SiO2'                  # Name of the material
SiO2.Young_modulus = 71.2*10**9     # Young's modulus (Pa)
SiO2.Poisson_ratio = 0.16           # Poisson ratio
SiO2.refractive_index = 1.5         # Refractive index
SiO2.mass_density = 2220            # Mass density (kg/m^3)

Ta2O5 = Materials()
Ta2O5.name = 'Ta2O5'                # Name of the material
Ta2O5.Young_modulus = 121*10**9     # Young's modulus (Pa)
Ta2O5.Poisson_ratio = 0.3           # Poisson ratio
Ta2O5.refractive_index = 2          # Refractive index
Ta2O5.mass_density = 7400           # Mass density (kg/m^3)


In [ ]:
class Substrate:
    def __init__(self, material, thickness):
        self.name = material.name
        self.Young_modulus = material.Young_modulus
        self.Poisson_ratio = material.Poisson_ratio
        self.refractive_index = material.refractive_index   # Refractive index of the substrate material
        self.mass_density = material.mass_density
        self.thickness = thickness

class Materials:
    pass



In [ ]:

import material_parameters
from Substrate import Substrate
from Layers import Layers

# Define the substrate
# Specify the substrate material. Use the following syntax: material_parameters.material
# Supply the thickness of the substrate in meters [m]
substrate = Substrate(material_parameters.fused_SiO2, 1e-3)


# Define the layers
# Input the material composing each layer. Use the following syntax: material_parameters.material
# Supply the optical thickness for each layer (expressed as a fraction of the wavelength of light)
layer_1 = Layers(material_parameters.SiO2, 0.25)
layer_2 = Layers(material_parameters.Ta2O5, 0.25)

# Define a stack of coating layers (put layers in order starting from the incidence medium)
layers = [layer_1, layer_2]

